the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


06/14/19:
# prior to this point my pipeline:
1. first median standardized
2. aggregated
3. converted to 2class
4. train/test split
5. imputed


# a big change will happen in this notebook, I will first:
1. convert to two class (c-/abshort &c+/ablong)
2. split the train and test set
3. median standardize
4. aggregate
5. impute

i do 1-3 BEFORE I AGGREGATE and median standardize, as some information may be leaking from the train/test set as i had it previously.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
from sklearn.model_selection import train_test_split
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # This is added back by InteractiveShellApp.init_path()


In [2]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path, categorical

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 1.09 s


In [3]:
len(patients)

15412

time: 2.68 ms


In [4]:

def save_df(df, df_name='default', add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    
    save_path= str(repository_path)+'/data/cleaned_merged_agg/'
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_cleaned_merged_agg.csv' %(date, df_name)))

time: 10.1 ms


In [5]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df


time: 1.59 ms


In [6]:
#importing cleaned_merged big_df
allFiles = glob.glob(str(repository_path)+ '/data/cleaned_merged/%s/'%(folder) + "{}_*.csv".format(date))

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

i=0
for name in df_list:
    big_df = pd.read_csv(allFiles[i],  index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 14 s


In [7]:
#quick housekeeping addition to accomidate older generated data
if len(big_df.loc[big_df['label']=="pao2/fio2",'label'])>1:
    big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2ratio"

time: 391 ms


In [8]:
big_df.head()

,icustay_id,t_0,label,value,delta,uom,source,subject_id
1124778,200001.0,2181-11-26,bands,absent,0 days 00:00:00.000000000,y/n,labs,55973
73154,200001.0,2181-11-26,pco2,absent,0 days 00:00:00.000000000,y/n,bg_ART,55973
15464,200001.0,2181-11-26,any_vasoactives,0.0,0 days 00:00:00.000000000,y/n,all_vaso,55973
14358,200001.0,2181-11-26,cancer_elix,0.0,0 days 00:00:00.000000000,y/n,cancer_elix,55973
27523,200001.0,2181-11-26,daily_sofa,5,0 days 00:00:00.000000000,daily_sofa_score,sofa,55973


time: 17.6 ms


In [9]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
any_vasoactives,19633,2,0.0,13144
nitrite,19633,2,Neg/Not_tested,19049
rrt,19633,2,0.0,17845
norepinephrine,19633,2,0.0,15737
gender,19633,2,M,10753
epinephrine,19633,2,0.0,19166
dopamine,19633,2,0.0,18468
vasopressin,19633,2,0.0,19061
o2_flow,19633,2,0.0,18523


time: 2.2 s


## initial data prep
* convert to two class

In [10]:
#convert to two class
final_pt_df2['final_bin'].unique()
two_classes=['C_neg/A_partial','C_pos/A_full']
two_class_icu=final_pt_df2.loc[final_pt_df2.loc[:,"final_bin"].isin(two_classes),['icustay_id','subject_id','final_bin']]


time: 9.05 ms


In [11]:
big_df=big_df.loc[big_df['icustay_id'].isin(list(two_class_icu['icustay_id'])),:].copy()

time: 1.49 s


In [12]:
len(big_df)
big_df['icustay_id'].nunique() 

10305

time: 58.2 ms


### roundabout way of sampling train/test set so that each subject is only in either train or test:
* To ensure a single patient did not end up with samples in both training and testing sets, individual patients (subject_id's) are kept together when performing the stratified train and test split

In [13]:
# label each subject_id with the max of the two classes. 
two_class_pt=two_class_icu.copy()
two_class_pt['final_bin']=pd.factorize(two_class_pt['final_bin'])[0]
two_class_maxsub=two_class_pt.loc[two_class_pt.groupby('subject_id')['final_bin'].idxmax(),:]

time: 1.57 s


In [14]:
# 70/30 train/test set split with 12345=seed, splitting on max final bin of each SUBJECT_ID
train, test = train_test_split(two_class_maxsub, test_size=0.3, random_state=12345, 
                               stratify=two_class_maxsub['final_bin'])
# generate list of each SUBJECT_ID in each split
train_subject=list(train['subject_id'])
test_subject=list(test['subject_id'])

#filtering big_df on train subjects and test subjects to get my train/test splits.
big_df_train= big_df.loc[big_df.loc[:,'subject_id'].isin(train_subject),:].copy()
big_df_test= big_df.loc[big_df.loc[:,'subject_id'].isin(test_subject),:].copy()
del big_df

time: 1.46 s


## calc median/iqr of 'healthiest patients' for standardization
take all non-categorical variables for HEALTHY PATIENTS and calculate the median and IQR for them. then will use this to make z scores via:

$$Z=\frac{(X-\widetilde{X}_{-/short})}{(IQR_{-/short})}$$ where $\widetilde{X}_{-/short}$ is the median value of the patients with negative SSC and short duration EAT.


In [15]:
big_df_train[big_df_train['label']=='vent_recieved']['icustay_id'].nunique()

7135

time: 146 ms


In [16]:
big_df_train['icustay_id'].nunique()

7205

time: 43.9 ms


In [17]:
from parameters import continuous, onetime


time: 789 µs


In [18]:
def median_label_fxn(big_df):
    """ grabs the median and IQR values for all continuous variables based upon the values present in
    the last 24 hours of the c_neg/A_partial patients (AKA the healthiest patients at the healthiest timepoints) """
    
    global final_pt_df2
    healthy_pt=list(final_pt_df2[final_pt_df2['final_bin']=="C_neg/A_partial"]['icustay_id'])

    #filter to only healthy patients filter
    ##splitting big_df, making a copy and restricting it to all values that will be usd in standardizing
    healthy_df=big_df[big_df['label'].isin(continuous+onetime)].copy()
    healthy_df['value']= pd.to_numeric(healthy_df['value']) #converting to numeric 
    healthy_df=healthy_df[healthy_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt
    
    #finding the last 24 hours of each healthy patient.
    healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())
    healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
    healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()
    healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
    
    #calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
    median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
    median_label=median_label.rename(columns={'value':"median"})
    iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
    iqr_label=iqr_label.rename(columns={'value':"iqr"})
    median_label=pd.merge(median_label,iqr_label)  #final median df
    return(median_label)

def standardization_fxn(big_df):
    global continuous, onetime, vaso_active, ordinal, categorical
    
    median_label=median_label_fxn(big_df)
    
    ### dataformatting: convert all dtypes to a numeric type that pereserves nan. 
    #splitting categorical, ordinal and continuous
    big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

    #continuous and ordinal variables
    big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous),:].copy() 
    big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 
    
    ### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 
    big_noCat=pd.merge(big_noCat, median_label, how="left") 
    #loging values
    big_noCat['median']= np.log(big_noCat['median']+1.0)
    big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
    big_noCat['raw_value']=big_noCat['value']
    big_noCat['value']=np.log(big_noCat['value']+1.0)
    
    big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized  

    return(big_noCat, big_categorical)

time: 121 ms


In [19]:
noCat_df_train, cat_df_train = standardization_fxn(big_df_train)
noCat_df_test, cat_df_test = standardization_fxn(big_df_test)

time: 1min 56s


In [20]:
#####

time: 797 µs


In [21]:
#should be same number of col as len(categorical)
cat_df_train[cat_df_train['icustay_id']==200012.0].groupby('label')['value'].value_counts()#.head(10)

label            value         
any_vasoactives  0.0               1
bands            >10               1
cancer_elix      0.0               1
dobutamine       0.0               1
dopamine         0.0               1
epinephrine      0.0               1
ethnicity        asian             1
gender           F                 1
leukocyte        Neg/Not_tested    1
nitrite          Neg/Not_tested    1
norepinephrine   0.0               1
o2_flow          0.0               1
pco2             absent            1
phenylephrine    0.0               1
rrt              0.0               1
vasopressin      0.0               1
vent_recieved    Oxygen            1
Name: value, dtype: int64

time: 10.8 ms


In [22]:
noCat_df_train[noCat_df_train['label']=='lactate']

,icustay_id,t_0,label,value,delta,uom,source,subject_id,median,iqr,raw_value,standardize
3,200012.0,2153-12-23,lactate,0.832909,0 days 02:36:00.000000000,mmol/L,labs,28448,0.993252,0.916291,1.3,-0.174991
131,200014.0,2105-02-17,lactate,0.641854,0 days 01:08:00.000000000,mmol/L,labs,9514,0.993252,0.916291,0.9,-0.383500
373,200014.0,2105-02-17,lactate,0.693147,1 days 04:00:00.000000000,mmol/L,labs,9514,0.993252,0.916291,1.0,-0.327521
498,200033.0,2198-08-11,lactate,0.955511,0 days 05:16:00.000000000,mmol/L,labs,56369,0.993252,0.916291,1.6,-0.041188
540,200033.0,2198-08-11,lactate,0.832909,0 days 09:36:00.000000000,mmol/L,labs,56369,0.993252,0.916291,1.3,-0.174991
1462,200055.0,2179-03-17,lactate,0.916291,0 days 08:12:00.000000000,mmol/L,labs,25225,0.993252,0.916291,1.5,-0.083992
1693,200059.0,2198-02-10,lactate,0.875469,0 days 01:56:00.000000000,mmol/L,labs,28494,0.993252,0.916291,1.4,-0.128543
1697,200059.0,2198-02-10,lactate,1.098612,0 days 03:34:00.000000000,mmol/L,labs,28494,0.993252,0.916291,2.0,0.114986
1702,200059.0,2198-02-10,lactate,0.693147,0 days 05:08:00.000000000,mmol/L,labs,28494,0.993252,0.916291,1.0,-0.327521
1741,200059.0,2198-02-10,lactate,0.832909,0 days 07:40:00.000000000,mmol/L,labs,28494,0.993252,0.916291,1.3,-0.174991


time: 1.59 s


# aggregation1:
### clincally guided min/max

In [23]:
from parameters import low_value, hi_value, both_value

important_onetime=['yearsold','weight'] #pco2 and bands are now in categorical 12-12-19


time: 16 ms


In [24]:
noCat_df_train['label'].unique()#noCat_df_train['label']=='any_vasoactives'

array(['daily_sofa', 'pao2fio2ratio', 'sum_elix', 'lactate', 'mingcs',
       'diasbp', 'heartrate', 'meanartpress', 'resprate', 'spo2', 'sysbp',
       'temperature', 'hemoglobin', 'platelet', 'wbc', 'calcium',
       'glucose', 'ph', 'bicarbonate', 'bun', 'chloride', 'creatinine',
       'inr', 'potassium', 'ptt', 'sodium', 'bilirubin'], dtype=object)

time: 217 ms


### running the min/max aggregations.



In [25]:
def clin_agg(big_noCat, big_cat, big_df, values="standardize"):
    """
    clincally guided aggregations.
    
    values= choose here if want to use standardization or raw values.
    
    note: getting two minor errors, could use some cleaning up at later date.
    """
    global hi_value, low_value, both_value, important_onetime
    #max aggregation for selected variables
    big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
    table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
    
    #min aggregation for selected variables
    big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
    table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)
    
    #max&min aggregation for selected variables
    big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
    table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)
    
    #first left join all different continuous aggregations together. 
    worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
    worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
    
    ### formatting categorical to wide format to match the tables/worst_df
    big_cat= big_cat.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per icustay per time. 
    
    ## merging the categorical and aggregated dataframes together. 
    worst_df=pd.merge(worst_df, big_cat, left_on='icustay_id', right_on='icustay_id',how='left') #using max/min aggregates 

    #adding important one_time values to final aggregated
    agg_remaining= big_df.loc[big_df.loc[:,'label'].isin(important_onetime),:]
    agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
    agg_table2.columns = agg_table2.columns.get_level_values(1)
    agg_table2=agg_table2.reset_index()
    agg_table2.head()#.rename(columns={})

    worst_df=pd.merge(worst_df, agg_table2, how='left')
    return(worst_df)


time: 141 ms


In [26]:
worst_df_train=clin_agg(noCat_df_train, cat_df_train,big_df_train, values="standardize")

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


time: 5.85 s


In [27]:
worst_df_test=clin_agg(noCat_df_test, cat_df_test, big_df_test, values="standardize")

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


time: 2.48 s


In [28]:
worst_df_train.head()

,icustay_id,bilirubin,bun,chloride,creatinine,daily_sofa,glucose,heartrate,inr,lactate,...,nitrite,norepinephrine,o2_flow,pco2,phenylephrine,rrt,vasopressin,vent_recieved,weight,yearsold
0,200012.0,NaN,NaN,NaN,NaN,-0.430677,NaN,0.072874,NaN,-0.174991,...,Neg/Not_tested,0.0,0.0,absent,0.0,0.0,0.0,Oxygen,51.2,32.9889590780034
1,200014.0,NaN,0.040770,0.037089,-0.152003,-0.178747,-0.024455,0.017360,-0.132111,-0.327521,...,Neg/Not_tested,0.0,0.0,<50,0.0,0.0,0.0,Mech,62.0,84.7281861738535
2,200033.0,NaN,-0.082229,0.012702,-0.234465,-0.430677,0.233006,0.117519,-0.132111,-0.041188,...,Neg/Not_tested,0.0,0.0,absent,1.0,0.0,0.0,Mech,74.0,67.1450990252744
3,200036.0,NaN,0.119501,0.020977,0.000000,-0.430677,0.271626,0.127153,0.126488,NaN,...,Neg/Not_tested,0.0,0.0,absent,0.0,0.0,0.0,Oxygen,79.0,74.934136467919
4,200055.0,-0.345426,0.198543,-0.021991,2.307429,0.503859,0.113341,0.070037,-0.132111,-0.083992,...,Neg/Not_tested,0.0,0.0,absent,0.0,1.0,0.0,None,56.0,31.593199704667


time: 30.5 ms


In [29]:
worst_df_train.head()

,icustay_id,bilirubin,bun,chloride,creatinine,daily_sofa,glucose,heartrate,inr,lactate,...,nitrite,norepinephrine,o2_flow,pco2,phenylephrine,rrt,vasopressin,vent_recieved,weight,yearsold
0,200012.0,NaN,NaN,NaN,NaN,-0.430677,NaN,0.072874,NaN,-0.174991,...,Neg/Not_tested,0.0,0.0,absent,0.0,0.0,0.0,Oxygen,51.2,32.9889590780034
1,200014.0,NaN,0.040770,0.037089,-0.152003,-0.178747,-0.024455,0.017360,-0.132111,-0.327521,...,Neg/Not_tested,0.0,0.0,<50,0.0,0.0,0.0,Mech,62.0,84.7281861738535
2,200033.0,NaN,-0.082229,0.012702,-0.234465,-0.430677,0.233006,0.117519,-0.132111,-0.041188,...,Neg/Not_tested,0.0,0.0,absent,1.0,0.0,0.0,Mech,74.0,67.1450990252744
3,200036.0,NaN,0.119501,0.020977,0.000000,-0.430677,0.271626,0.127153,0.126488,NaN,...,Neg/Not_tested,0.0,0.0,absent,0.0,0.0,0.0,Oxygen,79.0,74.934136467919
4,200055.0,-0.345426,0.198543,-0.021991,2.307429,0.503859,0.113341,0.070037,-0.132111,-0.083992,...,Neg/Not_tested,0.0,0.0,absent,0.0,1.0,0.0,None,56.0,31.593199704667


time: 25.5 ms


In [30]:
worst_df_train['pao2fio2ratio'].describe()

count    7205.000000
mean        0.100005
std         0.145107
min        -0.602952
25%        -0.020080
50%         0.208261
75%         0.208261
max         0.445545
Name: pao2fio2ratio, dtype: float64

time: 6.71 ms


In [31]:
worst_df_train['vasopressin'].value_counts()

0.0    7057
1.0     148
Name: vasopressin, dtype: int64

time: 5.84 ms


In [32]:
#worst_df_train['vasopressin'].value_counts()

time: 6.1 ms


In [33]:
worst_df_train['bands'].value_counts()

absent    6156
<10        687
>10        362
Name: bands, dtype: int64

time: 6.35 ms


In [34]:
worst_df_train['pco2'].value_counts()

absent    4699
<50       1740
>50        766
Name: pco2, dtype: int64

time: 19.1 ms


In [35]:
save_df(worst_df_train, 'train')
save_df(worst_df_test, 'test')

time: 717 ms


In [36]:
del worst_df_train, worst_df_test

time: 6 ms
